## Manual Phase Correction 

This script provides an interactive interface for manually splitting synchronized data into two distinct phases, the Reach&Grasp phase and Lift&Hold phase, cropping also corresponding video files accordingly. Utilizing ipywidgets, users can select participants, matches (trial Nr.), and camera files, input specific timepoints to define phase boundaries, visualize velocity data for accurate splitting, and automatically save the segmented data and videos.

- **Data Loading & Sorting:** Automatically identifies and sorts match-related data and camera files based on filenames.
- **Interactive Selection:** Users can select participants, choose specific matches, and pick relevant camera files through dropdown menus.
- **Velocity Visualization:** Plots smoothed X and Y velocities with user-defined phase timepoints to assist in accurate phase determination.
- **Manual Phase Input:** Allows users to input custom timepoints (`t0`, `t2`, `t3`) to define the start and end of each phase.
- **Automated Data Saving:** Saves the segmented phase data (`time`, `kin`, `myo`, `otb`) as `.npy` files for further analysis.
- **Video Cropping:** Automatically crops the selected video into Phase 1 and Phase 2 segments based on the input timepoints, saving the results for easy access.


In [1]:
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import butter, filtfilt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --------------------------------------------------------------------
# Utility Functions (Updated for Correct File Patterns)
# --------------------------------------------------------------------

def parse_match_num_from_filename(fullpath: str):
    """
    Extracts the match number from filenames like 'match_10_cropped_camera_0.avi'.
    Returns the match number as an integer or None if the pattern doesn't match.
    """
    filename = os.path.basename(fullpath)
    pattern = r"match_(\d+)_cropped_camera_(\d+)\.avi"
    m = re.match(pattern, filename)
    if not m:
        return None
    match_num = int(m.group(1))
    return match_num

def get_sorted_camera_files(camera_base_dir: str):
    """
    Searches for camera files matching 'match_*_cropped_camera_*.avi' in the specified directory.
    Extracts the match number from each filename.
    Returns a list of tuples: (camera_file_path, match_num).
    """
    pattern = os.path.join(camera_base_dir, "**", "match_*_cropped_camera_*.avi")
    cam_files = glob.glob(pattern, recursive=True)
    results = []
    for cf in cam_files:
        match_num = parse_match_num_from_filename(cf)
        if match_num is None:
            print(f"[WARN] Could not extract match number from filename: {cf}")
            continue
        results.append((cf, match_num))
    # Sort by match_num
    results.sort(key=lambda x: x[1])
    return results

def parse_match_number(fname: str):
    """
    For 'match_10_time.npy', returns int(10).
    If no match, returns None.
    """
    pattern = r"match_(\d+)_time\.npy"
    m = re.match(pattern, fname)
    if not m:
        return None
    return int(m.group(1))

def get_sorted_match_files(sync_dir: str):
    """
    Glob all 'match_*_time.npy' in sync_dir.
    Parse the numeric match index.
    Sort ascending by match_num.
    Returns list of (time_file_path, match_num).
    """
    time_files = glob.glob(os.path.join(sync_dir, "match_*_time.npy"))
    parsed_list = []
    for tf in time_files:
        base_name = os.path.basename(tf)
        match_num = parse_match_number(base_name)
        if match_num is not None:
            parsed_list.append((tf, match_num))
    parsed_list.sort(key=lambda x: x[1])
    return parsed_list

def load_synchronized_data(sync_dir, match_prefix, hand_dims):
    """
    Loads time, kin, myo, otb from .npy files based on match_prefix.
    Returns (time_data, kin_data, myo_data, otb_data) or None if missing files.
    """
    time_file = os.path.join(sync_dir, match_prefix + "_time.npy")
    kin_file  = os.path.join(sync_dir, match_prefix + "_kin.npy")
    myo_file  = os.path.join(sync_dir, match_prefix + "_myo.npy")
    otb_file  = os.path.join(sync_dir, match_prefix + "_otb.npy")
    if not all(os.path.exists(f) for f in [time_file, kin_file, myo_file, otb_file]):
        print(f"[WARN] Missing files for {match_prefix}.")
        return None
    time_data = np.load(time_file)
    kin_data  = np.load(kin_file)
    myo_data  = np.load(myo_file)
    otb_data  = np.load(otb_file)
    if len(time_data) < 5:
        print(f"[WARN] Not enough frames in {match_prefix}.")
        return None
    if kin_data.shape[1] < hand_dims:
        print(f"[WARN] kin_data has fewer than {hand_dims} columns.")
        return None
    return time_data, kin_data, myo_data, otb_data

def heavy_smooth(signal, dt, cutoff_hz=1.4, order=4):
    """
    Zero-phase Butterworth lowpass filter with filtfilt.
    """
    fs = 1.0 / dt  # sampling rate in Hz
    nyquist = 0.5 * fs
    Wn = cutoff_hz / nyquist  # normalize cutoff to [0..1]
    b, a = butter(order, Wn, btype='low', analog=False)
    return filtfilt(b, a, signal)

# --------------------------------------------------------------------
# Plotting Function (Updated for Manual Phases)
# --------------------------------------------------------------------

def plot_debug_velocities_manual(
    match_prefix,
    time_data,
    x_vel_sm,
    y_vel_sm,
    vel_time,
    phase_times=None,
    out_dir=None
):
    """
    Plots velocities with optional phase timepoints and saves the plot.
    """
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.plot(vel_time, x_vel_sm, label="Avg X Vel (smoothed)")
    plt.ylabel("X Velocity [units/s]")
    if phase_times:
        t0, t2, t3 = phase_times
        plt.axvline(t0, color='green', linestyle='--', label='t0')
        plt.axvline(t3, color='red', linestyle='--', label='t3')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(vel_time, y_vel_sm, label="Avg Y Vel (smoothed)", color='tab:orange')
    plt.xlabel("Time (s)")
    plt.ylabel("Y Velocity [units/s]")
    if phase_times:
        t0, t2, t3 = phase_times
        plt.axvline(t0, color='green', linestyle='--', label='t0')
        plt.axvline(t2, color='magenta', linestyle='--', label='t2')
        plt.axvline(t3, color='red', linestyle='--', label='t3')
    plt.legend()

    plt.tight_layout()
    
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)  # Ensure the directory exists
        plot_filename = os.path.join(out_dir, f"{match_prefix}_debug_vel.png")
        plt.savefig(plot_filename)
        print(f"[SAVED] Plot saved to {plot_filename}")
    
    plt.show()

# --------------------------------------------------------------------
# Video Cropping Function (Manual)
# --------------------------------------------------------------------

def split_save_and_crop_video_manual(
    match_prefix,
    time_data, kin_data, myo_data, otb_data,
    phase_times,
    out_dir,
    camera_file,
    camera_out_dir
):
    """
    Saves Phase1 & Phase2 data as .npy,
    then crops the single camera_file into Phase1.avi and Phase2.avi
    based on manually provided phase_times.

    phase_times: tuple with (t0, t2, t3)
    """
    t0, t2, t3 = phase_times

    # Validate timepoints
    if not (t0 < t2 < t3):
        print("[ERROR] Timepoints must satisfy t0 < t2 < t3.")
        return

    # Convert times to indices
    i0 = np.searchsorted(time_data, t0, side='left')
    i2 = np.searchsorted(time_data, t2, side='left')
    i3 = np.searchsorted(time_data, t3, side='left')

    # Ensure indices are within bounds
    i0 = max(i0, 0)
    i2 = max(i2, i0)
    i3 = max(i3, i2)
    i3 = min(i3, len(time_data))

    # PHASE 1 => [i0, i2)
    t_phase1  = time_data[i0:i2]
    kin_p1    = kin_data[i0:i2]
    myo_p1    = myo_data[i0:i2]
    otb_p1    = otb_data[i0:i2]

    # PHASE 2 => [i2, i3)
    t_phase2  = time_data[i2:i3]
    kin_p2    = kin_data[i2:i3]
    myo_p2    = myo_data[i2:i3]
    otb_p2    = otb_data[i2:i3]

    p1_prefix = match_prefix + "_phase1"
    p2_prefix = match_prefix + "_phase2"

    # Save each phase
    np.save(os.path.join(out_dir, p1_prefix + "_time.npy"), t_phase1)
    np.save(os.path.join(out_dir, p1_prefix + "_kin.npy"),  kin_p1)
    np.save(os.path.join(out_dir, p1_prefix + "_myo.npy"),  myo_p1)
    np.save(os.path.join(out_dir, p1_prefix + "_otb.npy"),  otb_p1)

    np.save(os.path.join(out_dir, p2_prefix + "_time.npy"), t_phase2)
    np.save(os.path.join(out_dir, p2_prefix + "_kin.npy"),  kin_p2)
    np.save(os.path.join(out_dir, p2_prefix + "_myo.npy"),  myo_p2)
    np.save(os.path.join(out_dir, p2_prefix + "_otb.npy"),  otb_p2)

    print(f"[SAVED] {match_prefix} => Phase1: {len(t_phase1)} frames, Phase2: {len(t_phase2)} frames.")

    # Video cropping
    def time_to_frame(tsec, fps, total_frames):
        frame_idx = int(round(tsec * fps))
        return max(0, min(frame_idx, total_frames - 1))

    # Convert times to frames
    cap = cv2.VideoCapture(camera_file)
    if not cap.isOpened():
        print(f"[WARN] Could not open {camera_file}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_f = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    s1 = time_to_frame(t0, fps, total_f)
    e1 = time_to_frame(t2, fps, total_f)
    s2 = time_to_frame(t2, fps, total_f)
    e2 = time_to_frame(t3, fps, total_f)

    cam_name = os.path.basename(camera_file)
    cam_sub_name = cam_name.replace(".avi","")  # e.g., 'match_03_cropped_camera_0'
    cam_sub  = os.path.join(camera_out_dir, cam_sub_name)
    os.makedirs(cam_sub, exist_ok=True)

    # PHASE 1 => [s1, e1)
    if s1 < e1:
        p1_file = os.path.join(cam_sub, f"{match_prefix}_phase1_{cam_name}")
        out1 = cv2.VideoWriter(p1_file, fourcc, fps, (w,h))
        cap.set(cv2.CAP_PROP_POS_FRAMES, s1)
        for idx in range(s1, e1):
            ret, frame = cap.read()
            if not ret:
                break
            out1.write(frame)
        out1.release()
        print(f"[CROPPED] Phase1 video saved to {p1_file}")

    # PHASE 2 => [s2, e2)
    if s2 < e2:
        p2_file = os.path.join(cam_sub, f"{match_prefix}_phase2_{cam_name}")
        out2 = cv2.VideoWriter(p2_file, fourcc, fps, (w,h))
        cap.set(cv2.CAP_PROP_POS_FRAMES, s2)
        for idx in range(s2, e2):
            ret, frame = cap.read()
            if not ret:
                break
            out2.write(frame)
        out2.release()
        print(f"[CROPPED] Phase2 video saved to {p2_file}")

    cap.release()

# --------------------------------------------------------------------
# Interactive Manual Phase Splitting Function (Revised)
# --------------------------------------------------------------------

def manual_phase_splitting():
    BASE_DIR = r"C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data"
    SYNC_FOLDER   = "Synchronized Data"
    SPLIT_FOLDER  = "Synchronized Data split in Phases"
    CROPPED_VIDEOS_SUBFOLDER = "Cropped Videos"

    n_hand_landmarks = 21
    hand_dims = n_hand_landmarks * 3

    # Step 1: Select Participant
    participants = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
    if not participants:
        print("[ERROR] No participants found in BASE_DIR.")
        return

    participant_dropdown = widgets.Dropdown(
        options=participants,
        description='Participant:',
        disabled=False,
    )

    display(participant_dropdown)

    # Step 2: Load Matches Button
    matches_dropdown = widgets.Dropdown(
        options=[],
        description='Match Num:',
        disabled=False,
    )

    matches_button = widgets.Button(
        description='Load Matches',
        disabled=False,
        button_style='',
        tooltip='Click to load matches',
        icon='refresh'
    )

    display(matches_button, matches_dropdown)

    # Step 3: Load Cameras Button
    camera_dropdown = widgets.Dropdown(
        options=[],
        description='Camera File:',
        disabled=False,
    )

    camera_button = widgets.Button(
        description='Load Cameras',
        disabled=False,
        button_style='',
        tooltip='Click to load camera files',
        icon='refresh'
    )

    display(camera_button, camera_dropdown)

    # Step 4: Input Timepoints
    t0_text = widgets.FloatText(
        value=0.0,
        description='t0 (s):',
        disabled=False
    )
    t2_text = widgets.FloatText(
        value=0.0,
        description='t2 (s):',
        disabled=False
    )
    t3_text = widgets.FloatText(
        value=0.0,
        description='t3 (s):',
        disabled=False
    )

    submit_button = widgets.Button(
        description='Submit Timepoints',
        disabled=False,
        button_style='success',
        tooltip='Click to submit timepoints',
        icon='check'
    )

    output = widgets.Output()

    display(t0_text, t2_text, t3_text, submit_button, output)

    # Initialize a dictionary to map match_num to match_prefix
    match_num_to_prefix = {}

    # Function to update matches based on selected participant
    def update_matches(*args):
        selected_participant = participant_dropdown.value
        participant_dir = os.path.join(BASE_DIR, selected_participant)
        sync_dir = os.path.join(participant_dir, SYNC_FOLDER)
        if not os.path.isdir(sync_dir):
            with output:
                clear_output()
                print(f"[WARN] No Synchronized Data folder for {selected_participant}.")
            matches_dropdown.options = []
            camera_dropdown.options = []
            return
        sorted_matches = get_sorted_match_files(sync_dir)
        match_options = [f"Match {num}" for _, num in sorted_matches]
        match_nums = [num for _, num in sorted_matches]
        # Populate the mapping
        match_num_to_prefix.clear()
        for tf, num in sorted_matches:
            prefix = os.path.basename(tf).replace("_time.npy", "")
            match_num_to_prefix[num] = prefix
        if match_options:
            matches_dropdown.options = match_nums
        else:
            with output:
                clear_output()
                print(f"[INFO] No matched files found for {selected_participant}.")
            matches_dropdown.options = []
            camera_dropdown.options = []

    # Button click handler to load matches
    def on_matches_button_clicked(b):
        with output:
            clear_output()
            update_matches()
            display(matches_dropdown, camera_dropdown)

    matches_button.on_click(on_matches_button_clicked)

    # Function to update camera files based on selected match
    def update_cameras(*args):
        selected_participant = participant_dropdown.value
        selected_match_num = matches_dropdown.value
        participant_dir = os.path.join(BASE_DIR, selected_participant)
        sync_dir = os.path.join(participant_dir, SYNC_FOLDER)
        cropped_videos_dir = os.path.join(sync_dir, CROPPED_VIDEOS_SUBFOLDER)
        sorted_cam = get_sorted_camera_files(cropped_videos_dir)
        # Filter camera files matching the selected match_num
        camera_files = [cf for cf, num in sorted_cam if num == selected_match_num]
        camera_options = [os.path.basename(cf) for cf in camera_files]
        if camera_options:
            camera_dropdown.options = camera_files
        else:
            with output:
                clear_output()
                print(f"[INFO] No camera files found for match number {selected_match_num}.")
        return

    # Button click handler to load cameras
    def on_camera_button_clicked(b):
        with output:
            clear_output()
            update_cameras()
            display(camera_dropdown)

    camera_button.on_click(on_camera_button_clicked)

    # Step 5: Process and Crop Videos Button Click Handler
    def on_submit_button_clicked(b):
        with output:
            clear_output()
            selected_participant = participant_dropdown.value
            selected_match_num = matches_dropdown.value
            selected_cam_file = camera_dropdown.value

            if not selected_participant or not selected_match_num or not selected_cam_file:
                print("[ERROR] Please select a participant, match number, and camera file.")
                return

            participant_dir = os.path.join(BASE_DIR, selected_participant)
            sync_dir = os.path.join(participant_dir, SYNC_FOLDER)
            split_dir = os.path.join(participant_dir, SPLIT_FOLDER)
            os.makedirs(split_dir, exist_ok=True)
            plots_dir = os.path.join(split_dir, "Plots_Velocity_Debug")
            os.makedirs(plots_dir, exist_ok=True)
            camera_out_dir = os.path.join(split_dir, "Camera")
            os.makedirs(camera_out_dir, exist_ok=True)

            # Retrieve the correct match_prefix from the mapping
            match_prefix = match_num_to_prefix.get(selected_match_num)
            if not match_prefix:
                print(f"[ERROR] No match_prefix found for match number {selected_match_num}.")
                return

            # Load synchronized data
            data_tuple = load_synchronized_data(sync_dir, match_prefix, hand_dims)
            if data_tuple is None:
                print(f"[WARN] Data missing for {match_prefix}, cannot proceed.")
                return
            time_data, kin_data, myo_data, otb_data = data_tuple

            # Compute smoothed velocities for plotting
            dt = np.median(np.diff(time_data))
            if dt <= 0:
                print("[ERROR] Invalid time data.")
                return
            pos = kin_data[:, :n_hand_landmarks*3]
            vel = (pos[1:] - pos[:-1]) / dt
            x_inds = np.arange(0, 3*n_hand_landmarks, 3)
            y_inds = np.arange(1, 3*n_hand_landmarks, 3)
            raw_x_vel = np.mean(vel[:, x_inds], axis=1)
            raw_y_vel = np.mean(vel[:, y_inds], axis=1)
            cutoff_hz = 1.4
            x_vel_sm = heavy_smooth(raw_x_vel, dt, cutoff_hz=cutoff_hz, order=4)
            y_vel_sm = heavy_smooth(raw_y_vel, dt, cutoff_hz=cutoff_hz, order=4)
            vel_time = 0.5 * (time_data[:-1] + time_data[1:])

            # Plot velocities with current timepoints
            t0, t2, t3 = t0_text.value, t2_text.value, t3_text.value
            phase_times = (t0, t2, t3)
            plot_debug_velocities_manual(
                match_prefix,
                time_data,
                x_vel_sm,
                y_vel_sm,
                vel_time,
                phase_times=phase_times,
                out_dir=plots_dir  # Save plots to the correct directory
            )

            # Validate timepoints
            if not (0 <= t0 < t2 < t3 <= time_data[-1]):
                print("[ERROR] Timepoints must satisfy 0 <= t0 < t2 < t3 <= total_time.")
                return

            # Crop and save
            split_save_and_crop_video_manual(
                match_prefix,
                time_data, kin_data, myo_data, otb_data,
                phase_times,
                out_dir=split_dir,
                camera_file=selected_cam_file,
                camera_out_dir=camera_out_dir
            )


    submit_button.on_click(on_submit_button_clicked)

# --------------------------------------------------------------------
# Run the Interactive Manual Phase Splitting
# --------------------------------------------------------------------

manual_phase_splitting()


Dropdown(description='Participant:', options=('old Pilot', 'P(1)', 'P(2)', 'P(3)', 'P(4)', 'P(5)', 'P(6)', 'P(…

Button(description='Load Matches', icon='refresh', style=ButtonStyle(), tooltip='Click to load matches')

Dropdown(description='Match Num:', options=(), value=None)

Button(description='Load Cameras', icon='refresh', style=ButtonStyle(), tooltip='Click to load camera files')

Dropdown(description='Camera File:', options=(), value=None)

FloatText(value=0.0, description='t0 (s):')

FloatText(value=0.0, description='t2 (s):')

FloatText(value=0.0, description='t3 (s):')

Button(button_style='success', description='Submit Timepoints', icon='check', style=ButtonStyle(), tooltip='Cl…

Output()